In [ ]:
# Simplified setup for Colab - need this as colab resets environment every runtime 
from google.colab import drive
drive.mount('/content/drive')

# Install just the essentials
!pip install -q mujoco
!pip install -q stable-baselines3
!pip install -q gymnasium
!pip install -q wandb

# Set environment variables
import os
os.environ["MUJOCO_GL"] = "osmesa"
os.environ["PYOPENGL_PLATFORM"] = "osmesa"

print("Setup complete!")

In [ ]:
## Then we get the versions to confirm everything is correct
import mujoco
import torch
import stable_baselines3
import gymnasium

print(f"MuJoCo: {mujoco.__version__}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"SB3: {stable_baselines3.__version__}")
print(f"Gymnasium: {gymnasium.__version__}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

MuJoCo version: 3.3.5
PyTorch version: 2.8.0+cpu
CUDA available: False
Stable-Baselines3 version: 2.7.0
Gymnasium version: 1.2.0


In [ ]:
# Checking file paths
import sys
sys.path.append('/content/ucr_project')

# Test imports
try:
    from ucr_v0h_config import UCRv0HConfig
    from ucr_fall_env import UCRFallRecoveryEnv
    from rl_training_setup import UCRTrainer
    print(" All UCR modules imported successfully!")

    # Test model path
    import os
    model_path = "/content/model/mjcf/xml/v0H_pos.xml"
    if os.path.exists(model_path):
        print(f" Model file found: {model_path}")
    else:
        print(f"Model file missing: {model_path}")

    # Check mesh files
    mesh_path = "/content/model/mjcf/mesh"
    if os.path.exists(mesh_path):
        mesh_folders = [f for f in os.listdir(mesh_path) if os.path.isdir(os.path.join(mesh_path, f))]
        print(f"Mesh folders found: {mesh_folders}")

    print("Setup verification complete!")

except ImportError as e:
    print(f" Import error: {e}")
    print("Make sure all Python files are uploaded to /content/ucr_project/")

Upload your UCR v0H project files:
1. Upload all your .py files (ucr_fall_env.py, ucr_v0h_config.py, etc.)
2. Upload your model folder with the XML and mesh files
3. Run the cells below


The syntax of the command is incorrect.
The syntax of the command is incorrect.


In [ ]:
# Get T4-optimized config
config = UCRv0HConfig.get_training_config("stage1")

# Update the model path to match your setup
config["model_path"] = "/content/model/mjcf/xml/v0H_pos.xml"

# Optimize for T4 GPU (15.8GB memory)
config.update({
    "n_envs": 8,  # Good for T4
    "device": "cuda",
    "stage1_timesteps": 800000,  # Reasonable for Colab session
    "max_episode_steps": 400,
    "use_wandb": False,  # Skip for now
    "save_freq": 25000,
    "eval_freq": 15000,
})

print(f"Training config:")
print(f"  Environments: {config['n_envs']}")
print(f"  Timesteps: {config['stage1_timesteps']:,}")
print(f"  Device: {config['device']}")
print(f"  Model: {config['model_path']}")

# Test environment creation first
print("\nTesting environment creation...")
test_env = UCRFallRecoveryEnv(
    model_path=config["model_path"],
    max_episode_steps=100,
    training_stage="stage1"
)
obs, info = test_env.reset()
print(f"Environment test successful! Observation shape: {obs.shape}")
test_env.close()

# Initialize trainer
print("Initializing trainer...")
trainer = UCRTrainer(config)

# Start training
print("Starting Stage 1 training...")
print("This will take approximately 2-3 hours on T4 GPU")
trainer.train_stage1()